## پروژه اول درس هوش مصنوعی با موضوع الگوریتم های جستجوی آگاهانه و ناآگاهانه

### در این پروژه سعی میکنیم با استفاده از الگوریتم های زیر مساله ی یافتن راهی برای جمع آوری معجون ها از معبدی با شکل شطرنجی را حل کنیم

## BFS , IDS & A*

# Imports

In [183]:
##import numpy as np
import time
from copy import deepcopy
import queue as qu

# Body

## Load tests data

In [33]:
tests = []
file = open("test1.in")
tests.append(file.readlines())
file.close()
file = open("test2.in")
tests.append(file.readlines())
file.close()
file = open("test3.in")
tests.append(file.readlines())
file.close()

#### داده های تست ها را از فایل هایی که در پوشه پروژه قرار دارند برای تست برنامه لود می کنیم
#### با توجه به تست های موجود فرض می شود که تعداد سطر ها و ستون ها اعداد تک رقمی می باشند 

## Create tests maps

In [111]:
maps = []
#C = []
#K = []
for test in tests:
    num_of_row = int(test[0][0])
    num_of_col = int(test[0][2])
    curr_map = [['e'] * num_of_col for i in range(num_of_row)]
    #maps.append(temp_map)
    curr_c = int(test[1][0])
    #C.append(curr_c)
    curr_k = int(test[1][2])
    #K.append(curr_k)
    for i in range(0, curr_c):
        curr_map[int(test[2+i][0])][int(test[2+i][2])] = 'c'
    for i in range(0, curr_k):
        curr_map[int(test[2+curr_c+i][0])][int(test[2+curr_c+i][2])] = 'k'
    curr_d = int(test[2+curr_c+curr_k])
    for i in range(0, curr_d):
        curr_map[int(test[3+curr_c+curr_k+i][0])][int(test[3+curr_c+curr_k+i][2])] = 'd'
    maps.append(curr_map)


In [151]:
maps

[[['e', 'k', 'c', 'e'],
  ['e', 'e', 'd', 'e'],
  ['c', 'e', 'e', 'e'],
  ['e', 'e', 'e', 'e']],
 [['k', 'e', 'e', 'd'],
  ['e', 'k', 'c', 'd'],
  ['c', 'c', 'e', 'e'],
  ['e', 'k', 'e', 'e']],
 [['e', 'e', 'd', 'e', 'e', 'e'],
  ['e', 'd', 'c', 'e', 'e', 'c'],
  ['e', 'e', 'k', 'e', 'e', 'e'],
  ['d', 'e', 'e', 'e', 'e', 'c'],
  ['e', 'd', 'k', 'c', 'e', 'e'],
  ['e', 'c', 'e', 'e', 'e', 'e']]]

#### مشاهده می شود که نقشه های تست به درستی  لود شده اند

## Model Description

### Actions : Di up, Di down, Di right, Di left (for i in range num of doctors)

### Initial State : A doctor in (0, 0) and the map looks like the test files

### Goal State : Every doctor in (n-1, m-1) and no 'c' remain in the map

### Path cost : sum of moves (Actions) of all doctors (from Initial state to Goal state)

### State : position of elements in the map in a node represents the current state for this node

### Transition Model : اگر دکتری به خانه ای که دارای دارو است وارد شود آن خانه تبدیل به خانه ی خالی شده و یک دکتر در خانه ی پایین سمت چپ جدول ایجاد می شود . اگر دکتری وارد خانه ای دارای معجون شود آن خانه به خانه ی خالی تبدیل می شود . برای حرکت دکتر ها هم تنها در صورتی که خانه ی مورد نظر مجاز باشد و قابل رفتن باشد به آنجا می رود

## Data Structures

In [226]:
class Node:
    def __init__(self, curr_map, path_until_there, path_cost_until_there, curr_Drs):
        self.state = curr_map
        self.path = path_until_there
        self.path_cost = path_cost_until_there
        self.Drs = curr_Drs
        self.m = len(curr_map)
        self.n = len(curr_map[0])
    
    def Goal_test(self):
        for row in self.state:
            if 'c' in row:
                return False
        for dr in self.Drs:
            if dr[1] != self.m-1 or dr[2] != self.n-1:
                return False
        return True
    
    def expand(self, fifo):
        for dr in self.Drs:
            if dr[1]+1 < self.m:
                if self.state[dr[1]+1][dr[2]] != 'd':
                    temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    if temp_state[dr[1]+1][dr[2]] == 'k':
                        temp_drs.append([len(self.Drs), self.m-1, 0])
                    temp_state[dr[1]+1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path_until_there).append([dr[0], "Down"])
                    temp_node = Node(temp_state, temp_path, self.path_cost+1, temp_drs)
                    if temp_node.Goal_test():
                        return temp_node
                    fifo.append(temp_node)
            if dr[1] > 0:
                if self.state[dr[1]-1][dr[2]] != 'd':
                    temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    if temp_state[dr[1]-1][dr[2]] == 'k':
                        temp_drs.append([len(self.Drs), self.m-1, 0])
                    temp_state[dr[1]-1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path_until_there).append([dr[0], "Up"])
                    temp_node = Node(temp_state, temp_path, self.path_cost+1, temp_drs)
                    if temp_node.Goal_test():
                        return temp_node
                    fifo.append(temp_node)
            if dr[2]+1 < self.n:
                if self.state[dr[1]][dr[2]+1] != 'd':
                    temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    if temp_state[dr[1]][dr[2]+1] == 'k':
                        temp_drs.append([len(self.Drs), self.m-1, 0])
                    temp_state[dr[1]][dr[2]+1] = 'e'
                    temp_path = deepcopy(self.path_until_there).append([dr[0], "Right"])
                    temp_node = Node(temp_state, temp_path, self.path_cost+1, temp_drs)
                    if temp_node.Goal_test():
                        return temp_node
                    fifo.append(temp_node)
            if dr[2] < 0:
                if self.state[dr[1]][dr[2]-1] != 'd':
                    temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    if temp_state[dr[1]][dr[2]-1] == 'k':
                        temp_drs.append([len(self.Drs), self.m-1, 0])
                    temp_state[dr[1]][dr[2]-1] = 'e'
                    temp_path = deepcopy(self.path_until_there).append([dr[0], "Left"])
                    temp_node = Node(temp_state, temp_path, self.path_cost+1, temp_drs)
                    if temp_node.Goal_test():
                        return temp_node
                    fifo.append(temp_node)
                    
        return None

## Common Functions

In [227]:
##def Goal_test(node):
    

## Implementation with BFS

### این الگوریتم کامل است و همواره جواب بهینه را به دست می آورد ولی از نظر تعداد استیت های دیده شده و زمان اجرا بهینه نیست 

### در این پیاده سازی برای راحتی پیاده سازی از مدیریت استیت های تکراری صرف نظر شده است

In [228]:
maps_copy = deepcopy(maps)

In [229]:
fifo = qu.Queue()

In [230]:
fifo.put(Node(maps_copy[0], [], 0, [0, 0, 0]))

In [231]:
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo)
    if answer != None:
        print("answer found")
        print(answer.path)


TypeError: 'int' object is not subscriptable

int

## Implementation with IDS

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [185]:
maps_copy = deepcopy(maps)

## Implementation with A* with consistent heuristic

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [186]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.3 

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [187]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.5

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [188]:
maps_copy = deepcopy(maps)